### **Casas em Boston Regressão linear**

**Importando as bibliotecas**

In [219]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statistics as sts
import warnings
from scipy.stats import probplot,shapiro,spearmanr,pearsonr,kendalltau
from statsmodels import stats
from statsmodels.stats.diagnostic import lilliefors
import statsmodels
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


warnings.filterwarnings('ignore')

df = pd.read_csv('/Users/diogomonteiro/housing.csv')

### **Entendendo as colunas**

RM: É o número médio de cômodos entre os imoveis no bairro.

LSTAT: É a porcentagem de proprietários no bairro considerados de "classe baixa" (proletariado)

PTRATIO: É a razão entre estudantes e professores nas escolas de ensino fundamental e médio no bairro.

### **Coluna alvo**

MEDV: Valor médio das casas        

In [ ]:
# Classe para análisar as colunas e plotar gráficos de dispersão
class Analisar():
    def __init__(self,dados):
        self.df = dados

    def analise(self):
        for i in self.df.columns:
            if self.df[i].dtype == 'int64' or self.df[i].dtype == 'float64':
                print(f'\n Nome da coluna:\n {self.df[i].name}')
                print(f'\n Tipo da coluna:\n {self.df[i].dtype}')
                print(f'\n Tipo da coluna:\n {self.df.corr()}')
                print(f'\n Valor media da coluna:\n {round(self.df[i].mean(),2)}')
                print(f'\n Quantidade de linhas da coluna:\n {self.df[i].shape}')
                print(f'\n Valores unicos da coluna:\n {self.df[i].isnull().sum()}')
                print(f'\n Valores nulos da coluna:\n {self.df[i].describe()}')
                fig = px.box(self.df[i])
                fig.update_layout(xaxis=dict(showgrid=False),yaxis=dict(showgrid=False),title=f'Boxplot da coluna {i}',template='plotly_dark')
                fig.show()

    def dispersao(self):
        for e in self.df.columns:
          fig1 = px.scatter(self.df,x=self.df[e],y=self.df.MEDV,title='Correlação Linear',template='plotly_dark')
          fig1.show()

Analise = Analisar(df)
Analise.dispersao()

In [ ]:
Analise.analise()

In [ ]:
fig = px.scatter_matrix(df,dimensions=["RM", "LSTAT", "PTRATIO", "MEDV"])
fig.update_layout(height=800,width=800,title='Correlação Linear',template='plotly_dark')
fig.show()

### **Análise da Normalidade**

Gráfico QQ-Plot

In [ ]:
probplot(df.MEDV,dist='norm',plot=plt)
plt.style.use('dark_background')
plt.title('Normal Q-Q plot')
plt.show()

In [ ]:
df.columns

In [ ]:
fig1 = px.histogram(df,x='MEDV',nbins=50,template='plotly_dark')
fig1.update_layout(title_text='Média dos valores')
fig1.show()

In [ ]:
probplot(df.RM,dist='norm',plot=plt)
plt.style.use('dark_background')
plt.title('Normal Q-Q plot')
plt.show()

In [ ]:
fig2 = px.histogram(df,x='RM',nbins=60,template='plotly_dark')
fig2.update_layout(title_text='Média dos quartos')
fig2.show()

In [ ]:
probplot(df.LSTAT,dist='norm',plot=plt)
plt.style.use('dark_background')
plt.title('Normal Q-Q plot')
plt.show()

In [ ]:
fig3 = px.histogram(df,x='LSTAT', nbins=40,template='plotly_dark')
fig3.update_layout(title_text='Porcentagem de moradores de classe baixa')
fig3.show()

In [ ]:
probplot(df.PTRATIO,dist='norm',plot=plt)
plt.style.use('dark_background')
plt.title('Normal Q-Q plot')
plt.show()

In [ ]:
fig4 = px.histogram(df,x='PTRATIO',nbins=60,template='plotly_dark')
fig4.update_layout(title_text='Razão entre estudante e professores')
fig4.show()

### Teste de Shapiro-Wilk
Shapiro-Wilk pode ser usado no maximo 5000 linhas

Ho = distribuição normal: p > 0.05

Ha = distribuição != normal: p <= 0.05

In [ ]:
shapiro(df.MEDV)

In [ ]:
estatistica,p =shapiro(df.MEDV)
print('Shapiro-Wilk')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {p}')

In [ ]:
estatistica,p =shapiro(df.RM)
print('Shapiro-Wilk')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {p}')

In [ ]:
estatistica,p =shapiro(df.LSTAT)
print('Shapiro-Wilk')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {p}')

In [ ]:
estatistica,p =shapiro(df.PTRATIO)
print('Shapiro-Wilk')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {p}')

### **Resultado:**
Não a distribuição normal

--------

### **Teste lilliefors (Kolmogorov_Sminov)**
Lilliefors pode ser usado com mais de 5000 linhas

Ho = distribuição normal: p > 0.05

Ha = distribuição normal: p<= 0.05

In [ ]:
estatistica,p =statsmodels.stats.diagnostic.lilliefors(df.MEDV,dist='norm')
print('Lilliefors')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {p}')

In [ ]:
estatistica,p =statsmodels.stats.diagnostic.lilliefors(df.RM,dist='norm')
print('Lilliefors')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {p}')

In [ ]:
estatistica,p =statsmodels.stats.diagnostic.lilliefors(df.LSTAT,dist='norm')
print('Lilliefors')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {p}')

In [ ]:
estatistica,p =statsmodels.stats.diagnostic.lilliefors(df.PTRATIO,dist='norm')
print('Lilliefors')
print(f'Estatisca do teste: {round(estatistica,2)}')
print(f'P-valor: {p}')

### **Resultado:**
Confirmação que não a distribuição normal

--------

### **Correlação Linear**
Pearson (distribuição normal)

Spearman (distribuição não normal)

Kendall (distribuição não normal com quantidade pequena de amostras)

Ho = Não há correlação linear: p > 0.05

Ha = existe correlação linear: p <= 0.05

**Pearson**

In [ ]:
coef,p = pearsonr(df.MEDV,df.RM)
print('Pearson')
print(f'Coeficiente de correlação: {coef}')
print(f'p-valor: {p}')

**Spearman**

In [ ]:
coef,p = spearmanr(df.MEDV,df.RM)
print('Spearman')
print(f'Coeficiente de correlação: {coef}')
print(f'p-valor: {p}')

**Kendall**

In [ ]:
coef,p = kendalltau(df.MEDV,df.RM)
print('Kendall')
print(f'Coeficiente de correlação: {coef}')
print(f'p-valor: {p}')

In [ ]:
correlacoes = df.corr(method='spearman')
correlacoes

In [ ]:
plt.figure()
sns.heatmap(correlacoes,annot=True)
sns.set_theme(style='darkgrid')

### **Regressão Linear Simples**

##Valor médio (MEDV) em função do número de cômodos (RM)

In [ ]:
df.head(2)

### Separando a coluna LSTAT

In [ ]:
x1 = df.iloc[:,0:1].values
x1

Separando as colunas alvo

In [ ]:
y = df.iloc[:,3].values
y

Separando os dados em teste e treino

In [ ]:
x_treino,x_teste,y_treino,y_teste = train_test_split(x1,y, test_size=0.3, random_state=10,)

In [ ]:
x_treino.shape,y_treino.shape

In [ ]:
x_teste.shape,y_teste.shape

In [ ]:
reg_linear1 = LinearRegression()
reg_linear1.fit(x_treino,y_treino)

Intercepto (coeficiente linear)

In [ ]:
reg_linear1.intercept_

Coeficiente Angular

In [ ]:
reg_linear1.coef_

**Equação: Valor = -626510.26 + 173771.45 vezes o número cômodos**

Coeficiente de Determinação dados de treino

In [ ]:
reg_linear1.score(x_treino,y_treino)

In [ ]:
reg_linear1.score(x_teste,y_teste)

In [ ]:
previsoes_treino = reg_linear1.predict(x_treino)
previsoes_treino

In [ ]:
plt.scatter(y=y_treino,x=x_treino,color='orange', s=10, alpha=0.9)
x_plot = np.linspace(3,9)
plt.plot(x_plot,x_plot*reg_linear1.coef_ + reg_linear1.intercept_,color='r')
plt.style.use('dark_background')
plt.title('Reta de regressão')
plt.ylabel('Valor médio')
plt.xlabel('Quantidades média de cômodos')
plt.show()

In [ ]:
previsoes_teste = reg_linear1.predict(x_teste)
previsoes_teste

In [ ]:
y_teste

In [ ]:
plt.scatter(y=y_teste,x=x_teste, color='orange', s=10, alpha=0.9)
x_plot = np.linspace(3,9)
plt.plot(x_plot,x_plot*reg_linear1.coef_ + reg_linear1.intercept_,color='r')
plt.style.use('dark_background')
plt.title('Reta de regressão')
plt.ylabel('Valor médio')
plt.xlabel('Quantidades de cômodos')
plt.show()

Fazendo previsões para valores distintos

In [ ]:
valor_casa =reg_linear1.predict([[9]])
valor_casa

### **Resposta:**

Coeficiente esta muito baixo, isso esta causando erro na previsão dos valores das casas.

Esta dando erro nas previsão de casas com 1 à 3 quartos.

------------

### **Métricas de Desempenho**

Erro absoluto

In [ ]:
abs(y_teste - previsoes_teste).mean()

Erro médio absoluto

In [ ]:
mean_absolute_error(y_teste,previsoes_teste)

Erro quadrádico médio

In [ ]:
mean_squared_error(y_teste,previsoes_teste)

Raiz do erro quadrático médio

In [ ]:
np.sqrt(mean_squared_error(y_teste,previsoes_teste))

**Valor médio (MEDV) em função da classe social (RM)**

In [ ]:
df.head(2)

Separando a coluna (RM)

In [ ]:
x2 = df.iloc[:,0:1]

Separando a coluna alvo (MEDV)

In [ ]:
y = df.iloc[:,3].values

Preparando os dados treino e teste

In [ ]:
x_treino,x_teste,y_treino,y_teste = train_test_split(x2,y,test_size=0.3,random_state=10)


In [ ]:
x_treino.shape,y_treino.shape

In [ ]:
x_teste.shape,y_teste.shape

Treinando o modelo

In [ ]:
reg_linear2 = LinearRegression()
reg_linear2.fit(x_treino,y_treino)

Intercepto (coeficinte linear)

In [ ]:
reg_linear2.intercept_

Coeficiente Angular

In [ ]:
reg_linear2.coef_

**Equação: Valor = -626510.26 + 173771.45 x LSTAT(Números de cômodos)**

Coeficiente de Determinacão dados de treino

In [ ]:
reg_linear2.score(x_treino,y_treino)

Coeficiente de Determinacão dados de teste

In [ ]:
reg_linear2.score(x_teste,y_teste)

**Previsão**

In [ ]:
previsoes_treino1 = reg_linear2.predict(x_treino)
previsoes_treino1

In [ ]:
plt.scatter(y=y_treino,x=x_treino, color='orange', s=10, alpha=0.9)
x_plot = np.linspace(3,9)
plt.plot(x_plot,x_plot*reg_linear2.coef_ + reg_linear2.intercept_,color='r')
plt.style.use('dark_background')
plt.title('Reta de regressão')
plt.ylabel('Valor médio')
plt.xlabel('Quantidades de Proprietários de Classe baixa (%)')
plt.show()

In [ ]:
# Prevendo os dados de teste
previsoes_teste = reg_linear2.predict(x_teste)
previsoes_teste

In [ ]:
# Analisando o score
reg_linear2.score(x_teste,y_teste)

In [ ]:
plt.scatter(y=y_teste,x=x_teste, color='orange', s=10, alpha=0.9)
x_plot = np.linspace(3,9)
plt.plot(x_plot,x_plot*reg_linear2.coef_ + reg_linear2.intercept_,color='r')
plt.style.use('dark_background')
plt.title('Reta de regressão')
plt.ylabel('Valor médio')
plt.xlabel('Quantidades de Proprietários de Classe baixa (%)')
plt.show()

In [ ]:
# Fazendo previsões para valores distintos
valor_casa = reg_linear2.predict([[4]])
valor_casa

Métricas de Desempenho

In [ ]:
# Erro absoluto
abs(y_teste - previsoes_teste).mean()

In [ ]:
# Erro médio absoluto
mean_absolute_error(y_teste, previsoes_teste)

In [ ]:
# Erro quadrático médio
mean_squared_error(y_teste, previsoes_teste)

In [ ]:
# Raiz do erro quadrático médio
np.sqrt(mean_squared_error(y_teste, previsoes_teste))

### Valor médio (MEDV) em função da classe social (LSTAT)

In [ ]:
df.head(3)

Separando a coluna LSTAT

In [ ]:
x3 = df.iloc[:,1:2].values
x3

Separando coluna alvo (MEDV)

In [ ]:
y = df.iloc[:,3].values
y

Separando dados treino e teste

In [ ]:
x_treino,x_teste,y_treino,y_teste = train_test_split(x3, y, test_size = 0.3, random_state=10)

In [ ]:
x_treino.shape, y_treino.shape

In [ ]:
x_teste.shape, y_teste.shape

Treinando o modelo

In [ ]:
reg_linear3 = LinearRegression()
reg_linear3.fit(x_treino,y_treino)

Intercepto (coeficiente linear)

In [ ]:
reg_linear3.intercept_

Coeficiente Angular

In [ ]:
reg_linear3.coef_

**Equação: Valor = 681977.74 - 17263.75 x LSTAT(Números de cômodos)**

Coeficiente de Determinação dados de treino

In [ ]:
reg_linear3.score(x_treino,y_treino)

In [ ]:
previsoes_treino = reg_linear3.predict(x_treino)
previsoes_treino

In [ ]:
plt.scatter(y=y_treino,x=x_treino, color="orange", s=10, alpha=0.9)
x_plot = np.linspace(0,40)
plt.plot(x_plot, x_plot*reg_linear3.coef_+reg_linear3.intercept_, color='r')
plt.title('Reta de regressão')
plt.ylabel('Valor médio')
plt.xlabel('Quantidade de proprietários de Classe Média Baixa (%)')
plt.show()

In [ ]:
previsoes_teste = reg_linear3.predict(x_teste)
previsoes_teste

In [ ]:
y_teste

Coeficiente de Determinação dados de teste

In [ ]:
reg_linear3.score(x_teste,y_teste)

In [ ]:
plt.scatter(y=y_teste,x=x_teste, color="orange", s=10, alpha=0.9)
x_plot = np.linspace(2,35)
plt.plot(x_plot, x_plot*reg_linear3.coef_+reg_linear3.intercept_, color='r')
plt.title('Reta de regressão')
plt.ylabel('Valor médio')
plt.xlabel('Quantidade de proprietários de Classe Média Baixa (%)')
plt.show()

In [ ]:
# Fazendo previsões para valores distintos
valor_casa = reg_linear3.predict([[25]])
valor_casa

Métricas de Desempenho

In [ ]:
# Erro absoluto
abs(y_teste - previsoes_teste).mean()

In [ ]:
# Erro médio Absoluto
mean_absolute_error(y_teste, previsoes_teste)

In [ ]:
# Erro quadrático médio
mean_squared_error(y_teste, previsoes_teste)

In [ ]:
# Raiz do erro quadrático médio (RMSE)
np.sqrt(mean_squared_error(y_teste, previsoes_teste))

Validação Cruzada

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits = 15, shuffle=True, random_state = 5)

In [ ]:
# Criando o modelo
modelo = LinearRegression()
resultado = cross_val_score(modelo,x3,y,cv = kfold)
resultado

In [ ]:
# Usando a média e o desvio padrão
print(f'Coef.determinação média: {round(resultado.mean()*100.0,2)}%')

Avaliando a Regressão Linear Simples com Statsmodels

In [ ]:
df.head(2)

In [ ]:
x = df.iloc[:,1:2].values

In [ ]:
y = df.iloc[:,3].values

In [ ]:
# Criando o modelo
regressao = smf.ols('y ~ x', data = df).fit()

In [ ]:
residuos = regressao.resid
residuos

### Teste de Normalidade dos resídos

Ho = distribuição normal: p > 0.05

Ha = distribuição != normal : p <= 0.05

In [ ]:
estatistica, p = shapiro(residuos)
print('Shapiro-Wilk')
print(f'Estatítica de teste: {estatistica}')
print(f'p-valor: {p}')

In [ ]:
probplot(residuos, dist='norm', plot=plt)
plt.style.use('dark_background')
plt.title('Normal Q-Q plot - resíduos')
plt.show()

### **Análise da Homocedasticidade**

(Resíduos com variação constante)

In [ ]:
plt.scatter(y=residuos, x=regressao.predict(), color='orange')
plt.hlines(y=0, xmin=0, xmax=900000, color='red')
plt.ylabel('Resíduos')
plt.xlabel('Valores Preditos')
plt.show()

### Teste Breusch-Pagan (Homocedasticidade ou Heterocedasticidade)

Ho = existe homocedasticidade: p > 0.05

Ha = não existe homocedasticidade: p <= 0.05

In [ ]:
estatistica, p, f, fp = sms.het_breuschpagan (regressao.resid, regressao.model.exog)
print('Teste Breusch-Pagan')
print(f'Estatística de teste: {estatistica}')
print(f'p-valor: {p}')
print(f'f-valor: {f}')
print(f'f_p-valor: {fp}')

### **Outliers nos resíduos**
(entre -3 e 3)

In [ ]:
outliers = regressao.outlier_test()

In [ ]:
outliers.max()

In [ ]:
outliers.min()

Modelo não tem consistência

------

### **Modelo de Regressão Linear**

In [ ]:
print(regressao.summary())

### **Estatística t:**

Ho = coeficiente igual a zero: p> 0,05 (coeficiente não validado)

Ha = coeeficiente diferente de zero: p <= 0,05 (coeficiente validado)

**Rˆ2 ajustado = 0,578**

In [ ]:
coefs = pd.DataFrame(regressao.params)
coefs.columns = ['Coeficientes']
print(coefs)

**Equação: Valor = 684138,49 - 17759.04 x LSTAT**

In [ ]:
regressao.params

In [ ]:
df.head()

In [ ]:
regressao.predict()

In [ ]:
plt.scatter(y=df.MEDV,x=df.LSTAT, color="orange", s=30, alpha=0.9)
x_plot = np.linspace(0,40)
plt.plot(x_plot, x_plot*regressao.params[1]+regressao.params[0], color='r')
plt.title('Reta de regressão')
plt.ylabel('Valor da Casa')
plt.xlabel('LSTAT')
plt.show()

### **Regressão Linear Múltipla**

In [ ]:
df.head(2)

In [ ]:
independente = df.iloc[:,0:3].values
independente

In [ ]:
independente.shape

In [ ]:
dependente = df.iloc[:,3].values
dependente

In [ ]:
x_treino,x_teste,y_treino,y_teste = train_test_split(independente,dependente, test_size = 0.3, random_state = 0)

In [ ]:
x_treino.shape,x_teste.shape

In [ ]:
multipla = LinearRegression()
multipla.fit(x_treino, y_treino)

In [ ]:
multipla.intercept_

In [ ]:
multipla.coef_

Coeficente de Determinação dos dados de treino

In [ ]:
multipla.score(x_treino,y_treino)

Teste

In [ ]:
previsoes = multipla.predict(x_teste)
previsoes

Coeficiente de Determinação dos dados de teste

In [ ]:
multipla.score(x_teste,y_teste)

Fazendo previsões para o valores distintos

In [ ]:
valor_casa = multipla.predict([[8,25,15]])
print(f'Valor do imóvel: {valor_casa}')

**Métricas**

Erro médio Absoluto

In [ ]:
mean_absolute_error(y_teste, previsoes)

Erro quadrático médio

In [ ]:
mean_squared_error(y_teste, previsoes)

Raiz do erro quadrático médio

In [ ]:
np.sqrt(mean_squared_error(y_teste, previsoes))

**Validação Cruzada**

Separando os dados em folds

In [ ]:
kfold = KFold(n_splits = 15, shuffle=True, random_state = 5)

Criando o modelo

In [ ]:
modelo = LinearRegression()
resultado = cross_val_score(modelo, independente, dependente, cv = kfold)
resultado

Usando a média e o desvio padrão

In [ ]:
print(f'Coeficiente de Determinação Médio : {round(resultado.mean() * 100,2)}%')

**Regressão Linear Simples:** Rˆ2 = 0,57/0,60; RMSE = 99315,5; Rˆ2 Validação Cruzada: 55,97%

**Regressão Linear Múltipla:** Rˆ2 = 0,73/0,68; RMSE = 96087,3; Rˆ2 Validação Cruzada: 69,25%

### **Avaliando a Regressão Linear Múltipla com Statsmodels**

In [ ]:
df.head(2)

Criação do modelo

In [ ]:
modelo = smf.ols('MEDV ~ RM + LSTAT + PTRATIO', data = df).fit()

In [ ]:
residuos = modelo.resid
residuos

**Teste de Normalidade dos resíduos**


Ho = distribuição normal: p > 0.05

Ha = distribuição != normal: p <= 0.05

In [ ]:
estatistica, p = shapiro(residuos)
print(f'Estatística de teste: {estatistica}')
print(f'p_valor: {p}')

In [ ]:
probplot(residuos, dist='norm', plot=plt)
plt.title('Normal Q-Q plot - Resíduos')
plt.show()

**Análise da Homocedasticidade dos resíduos**

(resíduos com variação constante)

In [ ]:
plt.scatter(y=residuos, x=modelo.predict(), color='orange')
plt.hlines(y=0, xmin=0, xmax=900000, color='red')
plt.ylabel('Resíduos')
plt.xlabel('Valores Preditos')
plt.show()

### Teste Breusch-Pagan (Homocedasticidade ou heterocedasticidade)

Ho = ecxiste homocedasticidade: p > 0.05

Ha = não existe homocedasticidade: p <= 0.05

In [ ]:
estatistica, p, f, fp = sms.het_breuschpagan(modelo.resid, modelo.model.exog)
print(f'Estatística de teste: {estatistica}')
print(f'p-valor: {p}')
print(f'f-valor: {f}')
print(f'f_p-valor: {fp}')

**Outliers nos resíduos**

(entre -3 e 3)

In [ ]:
outliers = modelo.outlier_test()
outliers.max()

In [ ]:
outliers.min()

### **Ausência de Multicolinearidade**

Somente entre as variáveis independentes.

Considerada multicolinearidade quando r > 0.9

In [ ]:
variaveis = df[["RM",'LSTAT','PTRATIO']]

In [ ]:
variaveis.head()

In [ ]:
correlacoes = variaveis.corr(method='pearson')
sns.heatmap(correlacoes,annot=True, fmt=".2f", cmap='viridis');

### **Análise do modelo**

Intercept

p_valor para cada coeficiente < 0.05 (estatisticamente significativos).

Ajusted R-squared (explicação do modelo através dos dados).

p_valor da estatística f< 0.05 (valida o modelo de regressão).

In [ ]:
print(modelo.summary())

**Equação:** Valorda casa = 4.155e+05.Itercept + 8.657e+04.RM - 1.085e+04.LSTAT - 1.949e+04.PTRATIO

Criando uma coluna com previsão

In [ ]:
df['previsao'] = modelo.fittedvalues

In [ ]:
df.head()

In [ ]:
sns.lmplot(x='previsao', y='MEDV', data=df);

----

### **Regressão Polinomial**

Valor médio (MEDV) em função do número de cômodos (RM)

In [ ]:
df.head(2)

In [ ]:
x = df.iloc[:,0:1].values
x

In [ ]:
y = df.iloc[:,3].values
y

In [ ]:
# Separando dados de treino e teste
x_treino,x_teste,y_treino,y_teste = train_test_split(x,y, test_size = 0.3, random_state = 0)

In [ ]:
x_treino.shape, y_treino.shape

In [ ]:
x_teste.shape, y_teste.shape

In [ ]:
# Pré Processamento
grau_polinomial = PolynomialFeatures(degree=2)

In [ ]:
x_poly = grau_polinomial.fit_transform(x_treino)
x_poly

In [ ]:
polinomial = LinearRegression()

In [ ]:
polinomial.fit(x_poly, y_treino)

In [ ]:
previsoes_treino = polinomial.predict(x_poly)

In [ ]:
polinomial.coef_

In [ ]:
polinomial.intercept_

**Equação:**

Valor = 1640107 -568528.11xRM + 60092.59xRMˆ2

In [ ]:
numeros = np.linspace(3,9.84,342)
numeros

In [ ]:
numeros.shape

In [ ]:
valor = 1640107 -568528.11*numeros + 60092.59*numeros**2

In [ ]:
plt.scatter(x_treino,y_treino,c = 'orange')
plt.xlabel('Quantidade de cômodos')
plt.ylabel('Valores casas')
plt.title('Previsão dos valores das casas: dados de treino')
plt.plot(numeros,valor,color='red')
plt.show();

In [ ]:
# Fazendo previsões para valores distintos
comodos = 6
valor = 1640107 -568528.11*comodos + 60092.59*comodos**2
valor

In [ ]:
# Coeficiente de Determinação dados de treino
polinomial.score(x_poly,y_treino)

Teste

In [ ]:
numeros1 = np.linspace(3,8.88,147)
numeros1

In [ ]:
numeros1.shape

In [ ]:
valor1 = 1640107 -568528.11*numeros1 + 60092.59*numeros1**2
valor1

In [ ]:
plt.scatter(x_teste,y_teste,c = 'orange')
plt.xlabel('Quantidade de cômodos')
plt.ylabel('Valores casas')
plt.title('Previsão dos valores das casas: dados de teste')
plt.plot(numeros1,valor1,color='red')
plt.show();

In [ ]:
x_poly_teste = grau_polinomial.fit_transform(x_teste)
x_poly_teste

In [ ]:
polinomial_teste = LinearRegression()

In [ ]:
polinomial_teste.fit(x_poly_teste,y_teste)

In [ ]:
# Coeficiente de Determinação dados de teste
polinomial.score(x_poly_teste,y_teste)

In [ ]:
previsoes_teste = polinomial_teste.predict(x_poly_teste)

**Métricas**

In [ ]:
# Erro médio absoluto
mean_absolute_error(y_teste, previsoes_teste)

In [ ]:
# Erro quadrático médio
mean_squared_error(y_teste, previsoes_teste)

In [ ]:
# Raiz do erro quadrático médio (RMSE)
np.sqrt(mean_squared_error(y_teste, previsoes_teste))

### **Resultado:**
**Regressão Linear Simples:** Rˆ2 = 0,57/0,60; RMSE = 99315,5; Rˆ2 Validação Cruzada: 55,97%

**Regressão Linear Múltipla:** Rˆ2 = 0,73/0,68; RMSE = 96087,3; Rˆ2 Validação Cruzada: 69,25%

**Regressão Polinomial:** Rˆ2 = 0,59/0,54; RMSE = 114670,62

---

### **Regressão por vetores de suporte**

In [ ]:
df.head(2)

In [ ]:
independente = df.iloc[:,0:3].values
independente

In [ ]:
independente.shape

In [ ]:
dependente = df.iloc[:,3].values
dependente

In [ ]:
# Separando dados treino e teste
x_treino,x_teste,y_treino,y_teste = train_test_split(independente, dependente, test_size = 0.3, random_state = 0)

In [ ]:
x_treino.shape,y_treino.shape

In [ ]:
x_teste.shape,y_teste.shape

In [ ]:
# kernil = rbf, linear, polinomial (poly)
svr = SVR(kernel='rbf')
svr.fit(x_treino, y_treino)

In [ ]:
svr.score(x_treino,y_treino)

In [ ]:
svr.score(x_teste,y_teste)

In [ ]:
previsoes = svr.predict(x_teste)
previsoes

In [ ]:
mean_absolute_error(y_teste, previsoes)

### Padronização de escala

In [ ]:
x_scaler = StandardScaler()
x_treino_scaler = x_scaler.fit_transform(x_treino)
x_treino_scaler

In [ ]:
y_scaler = StandardScaler()
y_treino_scaler = y_scaler.fit_transform(y_treino.reshape(-1,1))
y_treino_scaler

In [ ]:
x_teste_scaler = x_scaler.transform(x_teste)
x_teste_scaler

In [ ]:
y_teste_scaler = y_scaler.transform(y_teste.reshape(-1,1))
y_teste_scaler

In [ ]:
svr1 = SVR(kernel='rbf')
#.ravel() é para retornar matriz 1D
svr1.fit(x_treino_scaler, y_treino_scaler.ravel())

In [ ]:
svr1.score(x_treino_scaler, y_treino_scaler)

### Teste

In [ ]:
svr1.score(x_teste_scaler, y_teste_scaler)

In [ ]:
previsoes_teste = svr1.predict(x_teste_scaler)
previsoes_teste

In [ ]:
y_teste_scaler

### Revertendo a transformação

In [ ]:
y_teste_inverse = y_scaler.inverse_transform(y_teste_scaler)
previsoes_inverse = y_scaler.inverse_transform(previsoes_teste.reshape(-1,1))

In [ ]:
y_teste_inverse

In [ ]:
previsoes_inverse

In [ ]:
# Erro médio Absoluto
mean_absolute_error(y_teste_inverse, previsoes_inverse)

In [ ]:
# Erro quadrático médio
mean_squared_error(y_teste_inverse, previsoes_inverse)

In [ ]:
# Raiz do erro quadrático médio (RMSE)
np.sqrt(mean_squared_error(y_teste_inverse, previsoes_inverse))

### Validação Cruzada

Padronização de escala

In [ ]:
x = StandardScaler()
independente_scaler = x.fit_transform(independente)
independente_scaler

In [ ]:
y = StandardScaler()
dependente_scaler = y.fit_transform(dependente.reshape(-1,1))
dependente_scaler

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits = 18, shuffle=True, random_state = 5)

#Criando o modelo
modelo = SVR(kernel = 'rbf')
resultado = cross_val_score(modelo, independente_scaler,dependente_scaler.ravel(), cv = kfold)
resultado

In [ ]:
print(f'Coeficiente de Determinação Médio: {round(resultado.mean() * 100,2)}%')

### **Resultado:**
**Regressão Linear Simples:** Rˆ2 = 0,57/0,60; RMSE = 99315,5; Rˆ2 Validação Cruzada: 55,97%

**Regressão Linear Múltipla:** Rˆ2 = 0,73/0,68; RMSE = 96087,3; Rˆ2 Validação Cruzada: 69,25%

**Regressão Polinomial:** Rˆ2 = 0,59/0,54; RMSE = 114670,62

**Regressão SVR:** Rˆ2 = 0,87/0,81; RMSE = 73422.69; Rˆ2 Validação Cruzada: 82,71%

---

### **Regressão com árvore de decisão**

In [ ]:
df.head(2)

In [ ]:
independente = df.iloc[:,0:3].values
independente

In [ ]:
independente.shape

In [ ]:
dependente = df.iloc[:, 3].values
dependente

In [ ]:
dependente.shape

In [ ]:
x_treino,x_teste,y_treino,y_teste = train_test_split(independente,dependente, test_size=0.3, random_state=0)

In [ ]:
x_treino.shape,y_treino.shape

In [ ]:
x_teste.shape,y_teste.shape

In [193]:
modelos = {
    'Árvore': DecisionTreeRegressor(max_depth=5,random_state=10),
    'Random': RandomForestRegressor(n_estimators = 60, criterion ='squared_error', max_depth = 5, random_state = 10 ),
    'xgboost': XGBRegressor(n_estimators = 70, max_depth=3,learning_rate=0.1, objective='reg:squarederror'),
    'catboost': CatBoostRegressor(iterations=100, learning_rate=0.1,depth = 5, random_state =10),
     'Decision': DecisionTreeRegressor(max_depth = 5, random_state=10)
}
class treino_teste():

  def __init__(self,modelos,x_treino,x_teste,y_treino,y_teste):
    self.modelos = modelos
    self.x_treino = x_treino
    self.x_teste = x_teste
    self.y_treino = y_treino
    self.y_teste = y_teste
    self.modelo_score = {}

  def treino(self):

    for nome, model in self.modelos.items():

      model.fit(self.x_treino,self.y_treino)

      resultado_treino = round(model.score(self.x_treino,y_treino) * 100,2)
      resultado_teste = round(model.score(self.x_teste,y_teste) * 100,2)

      self.modelo_score[nome] = ['{:.2f}%'.format(resultado_treino), '{:.2f}%'.format(resultado_teste)]

    md = pd.DataFrame(self.modelo_score,index=['Coeficiente treino','Coeficiente teste'])

    return md

  def previsao(self,modelo,x,y):

    previsao = modelo.predict(x)

    # Erro médio absoluto
    print(f'Erro médio absoluto: {round(mean_absolute_error(y_teste, previsao),2)}\n')

    # Erro quadrático médio
    print(f'\nErro quadrático médio: {round(mean_squared_error(y_teste, previsao),2)}\n')

    # Raiz do erro quadrático médio (RMSE)
    print(f'\nRaiz do erro quadrático médio (RMSE): {round(np.sqrt(mean_squared_error(y_teste, previsao)),2)}')

  def val_cruzada(self,mod,):

    kfold = KFold(n_splits = 15, shuffle = True, random_state = 5)

    resultado = cross_val_score(mod, independente, dependente, cv = kfold)

    #Usando a média e o desvio padrão
    print(f'Coeficiente de determinação médio: {round(resultado.mean() * 100,2)}%')

Treino = treino_teste(modelos,x_treino,x_teste,y_treino,y_teste)
Treino.treino()

NameError: name 'CatBoostRegressor' is not defined

Métricas

In [ ]:
# Árvore de decisão
Treino.previsao(modelo=modelos['Árvore'],x=x_teste,y=y_teste)

In [ ]:
# Random Forest
Treino.previsao(modelo=modelos['Random'],x=x_teste,y=y_teste)

In [ ]:
# XGBRegressor
Treino.previsao(modelo=modelos['xgboost'],x=x_teste,y=y_teste)

In [ ]:
# Catboost
Treino.previsao(modelo=modelos['catboost'],x=x_teste,y=y_teste)

Validação Cruzada


In [ ]:
# Árvore de decisão validação cruzada
Treino.val_cruzada(mod=modelos['Decision'])

In [ ]:
# Random Forest validação cruzada
Treino.val_cruzada(mod=modelos['Random'])

In [ ]:
# xgboost validação cruzada
Treino.val_cruzada(mod=modelos['xgboost'])

In [ ]:
# Catboost validação cruzada
Treino.val_cruzada(mod=modelos['catboost'])

### **Resultado:**
**Regressão Linear Simples:** Rˆ2 = 0,57/0,60; RMSE = 99315,5; Rˆ2 Validação Cruzada: 55,97%

**Regressão Linear Múltipla:** Rˆ2 = 0,73/0,68; RMSE = 96087,3; Rˆ2 Validação Cruzada: 69,25%

**Regressão Polinomial:** Rˆ2 = 0,59/0,54; RMSE = 114670,62

**Regressão SVR:** Rˆ2 = 0,87/0,81; RMSE = 73422.69; Rˆ2 Validação Cruzada: 82,71%

**Regressão árvore de decisão:** Rˆ2 = 0,91/0,83; RMSE = 71114,5; Rˆ2 Validação Cruzada: 74,60%.

**Regressão Random Forest:** Rˆ2 = 0,91/0,84; RMSE = 66729.33; Rˆ2 Validação Cruzada: 82,85%.

**Regressão xgboost:** Rˆ2 = 0,92/0,84; RMSE = 68006.28; Rˆ2 Validação Cruzada: 83,18%.

**Regressão lightgbm:** Rˆ2 = 0,90/0,82; RMSE = 72295.92; Rˆ2 Validação Cruzada: 82.33%.

**Regressão catboost:** Rˆ2 = 0,90/0,83; RMSE = 68937.28; Rˆ2 Validação Cruzada: 83.21%.

----

O melhor modelo de Regressão Linear foi o **Random Forest**

---